In [1]:
# Install Firebase Admin SDK
!pip install firebase-admin


In [2]:

import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os

# Initialize Firebase
cred = credentials.Certificate(r"C:\Users\abhi1\Desktop\Machine Learning\TARP_CROP_PREDICTION\tarp-eec36-firebase-adminsdk-81oum-fb569f5824.json")  # Path to Firebase service account key
firebase_admin.initialize_app(cred)
db = firestore.client()


In [3]:

# Load dataset and train model
file_path = r'C:\Users\abhi1\Desktop\Machine Learning\TARP_CROP_PREDICTION\Modified_Crop_recommendation.csv'  # Path to your dataset file
crop_data_modified = pd.read_csv(file_path)
X = crop_data_modified[['Temperature', 'Humidity', 'Rainfall', 'soil_moisture']]
y = crop_data_modified['Crop']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
accuracy = accuracy_score(y_test, model.predict(X_test))
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Function to fetch rainfall data from OpenWeatherMap API
def fetch_rainfall(api_key, lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    rainfall = data.get('rain', {}).get('1h', 0)  # Rainfall in the last 1 hour
    return rainfall

# Function to give advice to farmers based on conditions
def farmer_advice(temperature, soil_moisture, crop_type):
    crop_temp_ranges = {
        'rice': (20, 30),
        'wheat': (10, 25),
        # Add more crops and their ideal temperature ranges
    }
    moisture_threshold = 30
    advice = []
    if soil_moisture < moisture_threshold:
        advice.append("Soil moisture is low, please water the crops.")
    ideal_temp_range = crop_temp_ranges.get(crop_type, (0, 100))
    if temperature > ideal_temp_range[1]:
        advice.append("Temperature is high, use sprinklers to cool down the crops.")
    return advice

# Function to collect data, predict crop, and send advice to Firebase
def predict_and_update_firebase(api_key, lat, lon):
    # Fetch environmental data from Firebase
    doc_ref = db.collection("farm_conditions").document("latest")
    doc = doc_ref.get()
    if doc.exists:
        data = doc.to_dict()
        temperature = data.get("temperature")
        humidity = data.get("humidity")
        soil_moisture = data.get("soil_moisture")
    else:
        print("No farm data found.")
        return

    # Fetch rainfall from OpenWeatherMap
    rainfall = fetch_rainfall(api_key, lat, lon)

    # Predict crop
    predicted_crop = model.predict([[temperature, humidity, rainfall, soil_moisture]])[0]

    # Get advice
    advice = farmer_advice(temperature, soil_moisture, predicted_crop)

    # Update Firebase with prediction and advice
    results_ref = db.collection("farm_conditions").document("latest")
    results_ref.set({
        "recommended_crop": predicted_crop,
        "advice": advice,
        "timestamp": firestore.SERVER_TIMESTAMP
    })
    print("Prediction and advice have been updated to Firebase.")

# Set up OpenWeatherMap API key and location details
api_key = "104df45900851af1f47f3c5038342f90"  # Replace with your actual API key
latitude = 12.9716  # Replace with farm's latitude
longitude = 77.5946  # Replace with farm's longitude

# Call the function to predict and update Firebase
predict_and_update_firebase(api_key, latitude, longitude)


Model Accuracy: 91.36%


c:\Users\abhi1\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Prediction and advice have been updated to Firebase.
